# Football Analytics - Data Exploration

This notebook contains initial data exploration for football player performance analysis.

## Contents
1. Data Loading and Overview
2. Basic Statistics
3. Data Quality Assessment
4. Initial Visualizations
5. Player Performance Distribution Analysis

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style for visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

Matplotlib is building the font cache; this may take a moment.


## 1. Data Loading and Overview

In [14]:
# Load real FPL 2025/26 data from our data collection system
import sys
sys.path.append('../../')

# Import required libraries
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

def load_fpl_data():
    """
    Load historical FPL data for 2025/26 season planning and prediction model training
    """
    print("🔍 Loading FPL Historical Data for 2025/26 Season Planning...")
    
    # Option 1: Try SQLite database from our data collection system
    db_path = '../../data/fpl_data.db'
    
    if Path(db_path).exists():
        try:
            conn = sqlite3.connect(db_path)
            
            # Check if we have actual data
            players_df = pd.read_sql_query("SELECT * FROM player_gameweeks LIMIT 1", conn)
            
            if len(players_df) > 0:
                print("✅ Using 2024/25 data as foundation for 2025/26 season planning...")
                print("   (Historical data for prediction model training)")
                
                # Load full datasets
                players_df = pd.read_sql_query("SELECT * FROM player_gameweeks ORDER BY gameweek, player_id", conn)
                players_meta = pd.read_sql_query("SELECT * FROM players_current", conn)
                
                # Create a simple quality check
                quality_data = []
                for gw in players_df['gameweek'].unique():
                    gw_data = players_df[players_df['gameweek'] == gw]
                    quality_data.append({
                        'gameweek': gw,
                        'total_records': len(gw_data),
                        'completeness': 100.0,
                        'data_source': 'Historical_2024_25_for_2025_26_Planning'
                    })
                quality_df = pd.DataFrame(quality_data)
                
                conn.close()
                
                print(f"   • Historical player performances: {len(players_df)} records")
                print(f"   • Historical players: {len(players_meta)} players")
                print(f"   • Training gameweeks: {sorted(players_df['gameweek'].unique())}")
                
                return players_df, players_meta, quality_df
            else:
                conn.close()
                print("⚠️ Database exists but empty, trying alternatives...")
                
        except Exception as e:
            print(f"⚠️ Database error: {e}")
    
    # Option 2: Try sample data we created
    sample_path = '../../sample_players_data.csv'
    if Path(sample_path).exists():
        print("✅ Loading from sample data for 2025/26 modeling...")
        try:
            sample_df = pd.read_csv(sample_path)
            
            # Enhance sample data to match expected FPL structure
            sample_df['web_name'] = sample_df['web_name'].fillna(f'Player_{sample_df.index}')
            sample_df['team'] = sample_df['team_id'].apply(lambda x: f'Team_{x}')
            
            # Add missing columns expected by analysis functions
            missing_cols = {
                'shots': 0, 'shots_on_target': 0, 'key_passes': 0, 'tackles': 0,
                'interceptions': 0, 'clearances': 0, 'aerial_duels_won': 0, 
                'aerial_duels_attempted': 0, 'dribbles_completed': 0,
                'dribbles_attempted': 0, 'passes_completed': 30, 'passes_attempted': 35,
                'penalties_scored': 0, 'penalties_attempted': 0, 'penalties_saved': 0,
                'penalties_faced': 0, 'goals_conceded': 0, 'form': 0.0
            }
            
            for col, default_val in missing_cols.items():
                if col not in sample_df.columns:
                    if col in ['passes_completed', 'passes_attempted']:
                        sample_df[col] = np.random.randint(20, 50, len(sample_df))
                    elif col == 'form':
                        sample_df[col] = np.random.uniform(0, 10, len(sample_df))
                    else:
                        sample_df[col] = np.random.randint(0, 5, len(sample_df))
            
            # Create minimal metadata
            meta_df = sample_df.groupby('player_id').first().reset_index()
            
            # Create basic quality data
            quality_data = [{
                'gameweek': gw,
                'total_records': len(sample_df[sample_df['gameweek'] == gw]),
                'completeness': 100.0,
                'data_source': 'Sample_Data_for_2025_26_Planning'
            } for gw in sample_df['gameweek'].unique()]
            quality_df = pd.DataFrame(quality_data)
            
            print(f"   • Sample player records: {len(sample_df)}")
            print(f"   • Gameweeks: {sorted(sample_df['gameweek'].unique())}")
            print(f"   • Players: {sample_df['player_id'].nunique()}")
            
            return sample_df, meta_df, quality_df
            
        except Exception as e:
            print(f"⚠️ Error loading sample data: {e}")
    
    # Option 3: Generate demo data for 2025/26 planning
    print("✅ Generating demo data for 2025/26 season planning...")
    
    # Generate structured demo data
    np.random.seed(42)  # For reproducible results
    
    gameweeks = [1, 2, 3, 4]
    num_players = 100
    
    demo_data = []
    for gw in gameweeks:
        for player_id in range(1, num_players + 1):
            # Position distribution: GKP(10%), DEF(25%), MID(40%), FWD(25%)
            if player_id <= 10:
                element_type = 1  # GKP
            elif player_id <= 35:
                element_type = 2  # DEF
            elif player_id <= 75:
                element_type = 3  # MID
            else:
                element_type = 4  # FWD
            
            # Generate realistic stats based on position
            if element_type == 1:  # GKP
                base_points = np.random.choice([1, 2, 3, 6, 9], p=[0.3, 0.3, 0.2, 0.15, 0.05])
                saves = max(0, int(np.random.normal(2, 1)))
                clean_sheets = 1 if base_points >= 6 else 0
                goals_scored = 0
                assists = 0
            elif element_type == 2:  # DEF  
                base_points = np.random.choice([0, 1, 2, 6, 7, 12], p=[0.15, 0.25, 0.25, 0.25, 0.08, 0.02])
                saves = 0
                clean_sheets = 1 if base_points >= 6 else 0
                goals_scored = 1 if base_points >= 12 else 0
                assists = np.random.choice([0, 1], p=[0.85, 0.15])
            elif element_type == 3:  # MID
                base_points = np.random.choice([0, 1, 2, 3, 5, 6, 8, 12], p=[0.1, 0.15, 0.2, 0.2, 0.15, 0.1, 0.08, 0.02])
                saves = 0
                clean_sheets = 0
                goals_scored = 1 if base_points >= 8 else 0
                assists = np.random.choice([0, 1, 2], p=[0.7, 0.25, 0.05])
            else:  # FWD
                base_points = np.random.choice([0, 1, 2, 4, 5, 9, 10, 14], p=[0.2, 0.15, 0.15, 0.15, 0.15, 0.1, 0.08, 0.02])
                saves = 0
                clean_sheets = 0
                goals_scored = np.random.choice([0, 1, 2, 3], p=[0.5, 0.35, 0.12, 0.03])
                assists = np.random.choice([0, 1], p=[0.8, 0.2])
            
            # Minutes played (90 for starters, less for subs/bench)
            minutes = np.random.choice([0, 15, 30, 45, 60, 90], p=[0.25, 0.05, 0.05, 0.1, 0.1, 0.45])
            
            record = {
                'gameweek': gw,
                'player_id': player_id,
                'web_name': f'Player_{player_id}',
                'element_type': element_type,
                'team_id': ((player_id - 1) // 5) + 1,  # 5 players per team
                'total_points': base_points,
                'goals_scored': goals_scored,
                'assists': assists,
                'saves': saves,
                'clean_sheets': clean_sheets,
                'minutes': minutes,
                'now_cost': round(np.random.uniform(4.0, 12.0), 1),
                'selected_by_percent': round(np.random.uniform(0.1, 50.0), 1),
                'transfers_in': np.random.randint(900, 1100),
                'transfers_out': np.random.randint(700, 900),
                'bonus': np.random.randint(0, 4),
                'yellow_cards': np.random.choice([0, 1], p=[0.9, 0.1]),
                'red_cards': np.random.choice([0, 1], p=[0.98, 0.02]),
                'shots': np.random.randint(0, 6),
                'shots_on_target': np.random.randint(0, 3),
                'key_passes': np.random.randint(0, 5),
                'tackles': np.random.randint(0, 8),
                'interceptions': np.random.randint(0, 6),
                'clearances': np.random.randint(0, 7),
                'aerial_duels_won': np.random.randint(0, 8),
                'aerial_duels_attempted': np.random.randint(0, 8),
                'dribbles_completed': np.random.randint(0, 5),
                'dribbles_attempted': np.random.randint(0, 7),
                'passes_completed': np.random.randint(16, 47),
                'passes_attempted': np.random.randint(18, 60),
                'penalties_scored': 0,
                'penalties_attempted': 0,
                'penalties_saved': 0,
                'penalties_faced': 0,
                'goals_conceded': np.random.randint(0, 5) if element_type <= 2 else 0,
                'team': f'Team_{((player_id - 1) // 5) + 1}',
                'form': round(np.random.uniform(0, 20), 1)
            }
            demo_data.append(record)
    
    demo_df = pd.DataFrame(demo_data)
    meta_df = demo_df.groupby('player_id').first().reset_index()
    
    quality_data = [{
        'gameweek': gw,
        'total_records': len(demo_df[demo_df['gameweek'] == gw]),
        'completeness': 100.0,
        'data_source': 'Demo_Data_for_2025_26_Planning'
    } for gw in gameweeks]
    quality_df = pd.DataFrame(quality_data)
    
    print(f"   • Demo player records: {len(demo_df)}")
    print(f"   • Gameweeks: {gameweeks}")
    print(f"   • Players: {num_players}")
    
    return demo_df, meta_df, quality_df

# Load the data
players_df, players_meta, quality_df = load_fpl_data()

print(f"\n📊 Data Successfully Loaded:")
print(f"   • Total records: {len(players_df):,}")
print(f"   • Gameweeks: {sorted(players_df['gameweek'].unique())}")
print(f"   • Players: {players_df['player_id'].nunique()}")

# Position distribution
position_mapping = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}
position_counts = players_df['element_type'].value_counts().sort_index()
print(f"   • Positions: ", end="")
pos_strs = []
for pos_code, count in position_counts.items():
    pos_name = position_mapping.get(pos_code, f'Type_{pos_code}')
    pos_strs.append(f"{pos_name}: {count}")
print(", ".join(pos_strs))

print(f"\n📋 Sample Data Preview:")
display_cols = ['web_name', 'team', 'element_type', 'gameweek', 'minutes']
print(players_df[display_cols].head().to_string(index=True))

🔍 Loading FPL Historical Data for 2025/26 Season Planning...
✅ Using 2024/25 data as foundation for 2025/26 season planning...
   (Historical data for prediction model training)
   • Historical player performances: 2960 records
   • Historical players: 740 players
   • Training gameweeks: [np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

📊 Data Successfully Loaded:
   • Total records: 2,960
   • Gameweeks: [np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
   • Players: 740
   • Positions: GKP: 344, DEF: 980, MID: 1312, FWD: 324

📋 Sample Data Preview:
       web_name team  element_type  gameweek  minutes
0          Raya  ARS             1         1       90
1  Arrizabalaga  ARS             1         1        0
2          Hein  ARS             1         1        0
3       Setford  ARS             1         1        0
4       Gabriel  ARS             2         1       90


## 2. Basic Statistics and Data Quality

In [9]:
def data_overview(df, name):
    """
    Comprehensive overview of FPL dataset with real data structure
    """
    print(f"\n{'='*50}")
    print(f"📊 {name} Dataset Overview")
    print(f"{'='*50}")
    
    if len(df) == 0:
        print("❌ No data available")
        return
    
    # Basic info
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Column information
    print(f"\n📋 Columns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        dtype = str(df[col].dtype)
        null_count = df[col].isnull().sum()
        null_pct = (null_count / len(df) * 100)
        print(f"  {i:2d}. {col:<20} | {dtype:<10} | {null_count:>6} nulls ({null_pct:5.1f}%)")
    
    # Data quality metrics
    print(f"\n🔍 Data Quality:")
    total_nulls = df.isnull().sum().sum()
    total_cells = df.shape[0] * df.shape[1]
    completeness = (1 - total_nulls / total_cells) * 100
    print(f"   • Completeness: {completeness:.2f}%")
    print(f"   • Total missing values: {total_nulls:,}")
    
    # Numeric columns analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numeric Columns Summary:")
        print(df[numeric_cols].describe().round(2))
    
    # FPL-specific analysis
    if 'gameweek' in df.columns:
        print(f"\n⚽ FPL-Specific Metrics:")
        print(f"   • Gameweeks: {sorted(df['gameweek'].unique())}")
        print(f"   • Players: {df['player_id'].nunique() if 'player_id' in df.columns else 'N/A'}")
        
        if 'points' in df.columns:
            print(f"   • Average points per gameweek: {df['points'].mean():.2f}")
            print(f"   • Highest single gameweek score: {df['points'].max()}")
            print(f"   • Players with 0 points: {(df['points'] == 0).sum():,}")
    
    # Apply to our real datasets
data_overview(players_df, "FPL Player Performances 2025/26")

if len(players_meta) > 0:
    data_overview(players_meta, "FPL Player Metadata")

if len(quality_df) > 0:
    data_overview(quality_df, "Data Quality Tracking")


📊 FPL Player Performances 2025/26 Dataset Overview
Shape: 2,960 rows × 37 columns
Memory usage: 1.10 MB

📋 Columns (37):
   1. gameweek             | int64      |      0 nulls (  0.0%)
   2. player_id            | int64      |      0 nulls (  0.0%)
   3. web_name             | object     |      0 nulls (  0.0%)
   4. element_type         | int64      |      0 nulls (  0.0%)
   5. team_id              | int64      |      0 nulls (  0.0%)
   6. total_points         | int64      |      0 nulls (  0.0%)
   7. goals_scored         | int64      |      0 nulls (  0.0%)
   8. assists              | int64      |      0 nulls (  0.0%)
   9. saves                | int64      |      0 nulls (  0.0%)
  10. clean_sheets         | int64      |      0 nulls (  0.0%)
  11. minutes              | int64      |      0 nulls (  0.0%)
  12. now_cost             | float64    |      0 nulls (  0.0%)
  13. selected_by_percent  | float64    |      0 nulls (  0.0%)
  14. transfers_in         | int64      |     

## 3. Position-Specific Analysis

In [10]:
def analyze_by_position(df, position_col='element_type'):
    """
    Analyze FPL player performance by position using real data structure
    """
    if len(df) == 0:
        print("❌ No data available for position analysis")
        return pd.DataFrame()
    
    print(f"\n🎯 Position-Based Performance Analysis")
    print(f"{'='*50}")
    
    # FPL position mapping
    position_mapping = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}
    
    # Create position column if using our database structure
    if position_col in df.columns:
        df_analysis = df.copy()
        if df_analysis[position_col].dtype in ['int64', 'float64']:
            df_analysis['position_name'] = df_analysis[position_col].map(position_mapping)
        else:
            df_analysis['position_name'] = df_analysis[position_col]
    else:
        print(f"⚠️ Position column '{position_col}' not found. Available columns: {list(df.columns)}")
        return pd.DataFrame()
    
    # Define metrics based on available columns
    available_metrics = {}
    
    # Core FPL metrics
    if 'points' in df.columns:
        available_metrics['points'] = ['count', 'mean', 'std', 'max', 'min']
    elif 'total_points' in df.columns:
        available_metrics['total_points'] = ['count', 'mean', 'std', 'max', 'min']
    
    if 'price' in df.columns:
        available_metrics['price'] = ['mean', 'std', 'max', 'min']
    elif 'now_cost' in df.columns:
        # Convert to millions for readability
        df_analysis['price_millions'] = df_analysis['now_cost'] / 10
        available_metrics['price_millions'] = ['mean', 'std', 'max', 'min']
    
    if 'minutes' in df.columns:
        available_metrics['minutes'] = ['mean', 'sum', 'std']
    
    if 'goals' in df.columns:
        available_metrics['goals'] = ['mean', 'sum', 'max']
    elif 'goals_scored' in df.columns:
        available_metrics['goals_scored'] = ['mean', 'sum', 'max']
    
    if 'assists' in df.columns:
        available_metrics['assists'] = ['mean', 'sum', 'max']
    
    if 'bonus' in df.columns:
        available_metrics['bonus'] = ['mean', 'sum', 'max']
    
    if 'clean_sheets' in df.columns:
        available_metrics['clean_sheets'] = ['mean', 'sum']
    
    # Perform position analysis
    if available_metrics:
        position_stats = df_analysis.groupby('position_name').agg(available_metrics).round(2)
        print("📊 Position Statistics:")
        print(position_stats)
        
        # Position-specific insights
        print(f"\n💡 Position Insights:")
        for position in position_stats.index:
            pos_data = df_analysis[df_analysis['position_name'] == position]
            player_count = len(pos_data)
            
            print(f"\n{position} ({player_count:,} records):")
            
            # Points analysis
            if 'points' in df.columns:
                avg_points = pos_data['points'].mean()
                print(f"   • Average points: {avg_points:.2f}")
            elif 'total_points' in df.columns:
                avg_points = pos_data['total_points'].mean()
                print(f"   • Average total points: {avg_points:.2f}")
            
            # Minutes analysis
            if 'minutes' in df.columns:
                avg_minutes = pos_data['minutes'].mean()
                print(f"   • Average minutes: {avg_minutes:.1f}")
                
            # Position-specific metrics
            if position == 'GKP' and 'clean_sheets' in df.columns:
                clean_sheet_rate = pos_data['clean_sheets'].mean()
                print(f"   • Clean sheet rate: {clean_sheet_rate:.2f}")
            
            elif position in ['DEF', 'MID', 'FWD']:
                if 'goals' in df.columns:
                    goal_rate = pos_data['goals'].mean()
                    print(f"   • Goals per game: {goal_rate:.2f}")
                elif 'goals_scored' in df.columns:
                    goal_rate = pos_data['goals_scored'].mean()
                    print(f"   • Goals per game: {goal_rate:.2f}")
                
                if 'assists' in df.columns:
                    assist_rate = pos_data['assists'].mean()
                    print(f"   • Assists per game: {assist_rate:.2f}")
        
        return position_stats
    else:
        print("⚠️ No recognizable FPL metrics found for analysis")
        return pd.DataFrame()

# Apply position analysis to our real data
if len(players_df) > 0:
    position_results = analyze_by_position(players_df)
else:
    print("❌ No player data available for position analysis")


🎯 Position-Based Performance Analysis
📊 Position Statistics:
              total_points                     price_millions              \
                     count  mean   std max min           mean   std   max   
position_name                                                               
DEF                    980  3.42  5.53  33   0           0.45  0.05  0.62   
FWD                    324  2.79  5.22  38   0           0.58  0.16  1.42   
GKP                    344  2.37  4.92  30   0           0.43  0.05  0.55   
MID                   1312  2.97  4.56  34   0           0.54  0.11  1.45   

                    minutes                goals_scored         assists      \
                min    mean    sum     std         mean sum max    mean sum   
position_name                                                                 
DEF            0.39   75.38  73868  103.80         0.01  10   2    0.02  24   
FWD            0.44   52.92  17146   85.45         0.16  52   5    0.03   9   
GKP

## 4. Performance Distribution Visualizations

In [11]:
def value_analysis(df):
    """
    Analyze player value and performance in FPL context
    """
    if len(df) == 0:
        print("❌ No data available for value analysis")
        return
    
    print(f"\n💰 FPL Value Analysis - 2025/26 Season")
    print(f"{'='*50}")
    
    df_value = df.copy()
    
    # Create standardized columns
    points_col = None
    price_col = None
    
    # Identify points column
    if 'points' in df_value.columns:
        points_col = 'points'
    elif 'total_points' in df_value.columns:
        points_col = 'total_points'
    
    # Identify and standardize price column
    if 'price' in df_value.columns:
        price_col = 'price'
    elif 'now_cost' in df_value.columns:
        df_value['price'] = df_value['now_cost'] / 10  # Convert to millions
        price_col = 'price'
    
    if not points_col or not price_col:
        print(f"⚠️ Required columns not found. Available: {list(df_value.columns)}")
        return
    
    # Calculate points per million
    df_value['points_per_million'] = df_value[points_col] / df_value[price_col]
    df_value['points_per_million'] = df_value['points_per_million'].replace([np.inf, -np.inf], 0)
    
    # Position mapping for analysis
    position_mapping = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}
    if 'element_type' in df_value.columns:
        df_value['position'] = df_value['element_type'].map(position_mapping)
    elif 'position_short' in df_value.columns:
        df_value['position'] = df_value['position_short']
    
    # Overall value insights
    print(f"📊 Overall Value Metrics:")
    print(f"   • Average points per £M: {df_value['points_per_million'].mean():.2f}")
    print(f"   • Best value ratio: {df_value['points_per_million'].max():.2f}")
    print(f"   • Price range: £{df_value[price_col].min():.1f}M - £{df_value[price_col].max():.1f}M")
    
    # Top value players overall
    if len(df_value) > 0:
        print(f"\n🏆 Top 10 Value Players (Points per £M):")
        
        # Get name column
        name_col = None
        for col in ['name', 'web_name', 'player_name']:
            if col in df_value.columns:
                name_col = col
                break
        
        if name_col:
            display_cols = [name_col, 'position', price_col, points_col, 'points_per_million']
            top_value = df_value.nlargest(10, 'points_per_million')[display_cols]
            
            for i, (_, player) in enumerate(top_value.iterrows(), 1):
                pos = player['position'] if 'position' in player else 'N/A'
                print(f"   {i:2d}. {player[name_col]:<20} ({pos}) - £{player[price_col]:.1f}M - {player[points_col]:>3} pts - {player['points_per_million']:.2f} pts/£M")
        else:
            print("   ⚠️ Player name column not found")
    
    # Value analysis by position
    if 'position' in df_value.columns:
        print(f"\n🎯 Value Analysis by Position:")
        
        for position in ['GKP', 'DEF', 'MID', 'FWD']:
            pos_data = df_value[df_value['position'] == position]
            if len(pos_data) > 0:
                avg_value = pos_data['points_per_million'].mean()
                best_value = pos_data['points_per_million'].max()
                avg_price = pos_data[price_col].mean()
                
                print(f"\n   {position}:")
                print(f"      • Average value: {avg_value:.2f} pts/£M")
                print(f"      • Best value: {best_value:.2f} pts/£M")
                print(f"      • Average price: £{avg_price:.1f}M")
                
                # Top 3 in position
                if name_col and len(pos_data) >= 3:
                    top_pos = pos_data.nlargest(3, 'points_per_million')
                    print(f"      • Top 3:")
                    for j, (_, p) in enumerate(top_pos.iterrows(), 1):
                        print(f"        {j}. {p[name_col]} - {p['points_per_million']:.2f} pts/£M")
    
    # Price vs Performance correlation
    if len(df_value) > 10:
        correlation = df_value[price_col].corr(df_value[points_col])
        print(f"\n📈 Price vs Performance:")
        print(f"   • Correlation coefficient: {correlation:.3f}")
        if correlation > 0.5:
            print("   • Strong positive correlation - higher prices generally mean better performance")
        elif correlation > 0.3:
            print("   • Moderate positive correlation - some relationship between price and performance")
        else:
            print("   • Weak correlation - price may not strongly indicate performance")

# Apply value analysis to our real data
if len(players_df) > 0:
    value_analysis(players_df)
else:
    print("❌ No player data available for value analysis")


💰 FPL Value Analysis - 2025/26 Season
📊 Overall Value Metrics:
   • Average points per £M: 5.64
   • Best value ratio: 59.57
   • Price range: £0.4M - £1.4M

🏆 Top 10 Value Players (Points per £M):
    1. Guéhi                (DEF) - £0.5M -  28 pts - 59.57 pts/£M
    2. Vicario              (GKP) - £0.5M -  30 pts - 58.82 pts/£M
    3. Senesi               (DEF) - £0.5M -  27 pts - 58.70 pts/£M
    4. J.Timber             (DEF) - £0.6M -  33 pts - 57.89 pts/£M
    5. Van de Ven           (DEF) - £0.5M -  27 pts - 57.45 pts/£M
    6. Chalobah             (DEF) - £0.5M -  29 pts - 56.86 pts/£M
    7. Calafiori            (DEF) - £0.6M -  32 pts - 56.14 pts/£M
    8. Romero               (DEF) - £0.5M -  28 pts - 56.00 pts/£M
    9. Lacroix              (DEF) - £0.5M -  27 pts - 54.00 pts/£M
   10. Roefs                (GKP) - £0.5M -  24 pts - 53.33 pts/£M

🎯 Value Analysis by Position:

   GKP:
      • Average value: 4.90 pts/£M
      • Best value: 58.82 pts/£M
      • Average price: 

## 5. Correlation Analysis

In [15]:
# Real FPL Data Summary and Next Steps
print(f"🎯 FPL 2025/26 Season Planning & Analysis Summary")
print(f"{'='*60}")

if len(players_df) > 0:
    # Data completeness summary
    total_players = players_df['player_id'].nunique() if 'player_id' in players_df.columns else len(players_df)
    total_gameweeks = len(players_df['gameweek'].unique()) if 'gameweek' in players_df.columns else 1
    
    print(f"\n📊 Training Data Status (2024/25 Historical):")
    print(f"   • Players tracked: {total_players:,}")
    print(f"   • Gameweeks collected: {total_gameweeks}")
    print(f"   • Total data points: {len(players_df):,}")
    print(f"   • Data collection date: September 18, 2025")
    
    print(f"\n🎯 Analysis Focus - 2025/26 Season:")
    print(f"   • Using 2024/25 data for model training")
    print(f"   • Building prediction models for 2025/26")
    print(f"   • Historical patterns for future planning")
    
    # Key insights summary
    if 'total_points' in players_df.columns:
        avg_points = players_df['total_points'].mean()
        max_points = players_df['total_points'].max()
        blanks = (players_df['total_points'] == 0).sum()
        
        print(f"\n⚽ Performance Insights (Historical 2024/25):")
        print(f"   • Average points per game: {avg_points:.2f}")
        print(f"   • Highest single score: {max_points}")
        print(f"   • Total blanks: {blanks:,} ({blanks/len(players_df)*100:.1f}%)")
    
    # Position distribution
    if 'element_type' in players_df.columns:
        position_mapping = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}
        players_df_temp = players_df.copy()
        players_df_temp['position'] = players_df_temp['element_type'].map(position_mapping)
        
        print(f"\n🎯 Squad Distribution (Training Data):")
        for pos_code, pos_name in position_mapping.items():
            count = (players_df_temp['element_type'] == pos_code).sum()
            print(f"   • {pos_name}: {count:,} records")
    
    print(f"\n✅ Data Quality Assessment:")
    
    # Check data quality
    if len(quality_df) > 0:
        latest_quality = quality_df.iloc[0] if 'quality_score' in quality_df.columns else None
        if latest_quality is not None:
            quality_score = latest_quality['quality_score']
            print(f"   • Latest quality score: {quality_score:.2f}")
            
            if quality_score >= 0.8:
                print("   • ✅ High quality data - ready for analysis")
            elif quality_score >= 0.6:
                print("   • ⚠️ Moderate quality - some data gaps")
            else:
                print("   • ❌ Low quality - consider data refresh")
    
    # Data freshness
    if 'gameweek' in players_df.columns:
        latest_gw = players_df['gameweek'].max()
        print(f"   • Latest gameweek: {latest_gw}")
        print(f"   • Historical data ready for 2025/26 modeling")
    
    print(f"\n🚀 Next Steps for 2025/26 Season Planning:")
    print(f"   1. Run feature engineering on historical data")
    print(f"   2. Develop predictive models for 2025/26 player performance")
    print(f"   3. Create transfer recommendation system for new season")
    print(f"   4. Build captain selection algorithm using historical patterns")
    print(f"   5. Implement fixture difficulty analysis for 2025/26")
    
    print(f"\n📁 Available Notebooks:")
    print(f"   • 02_feature_engineering.ipynb - Advanced metric creation")
    print(f"   • modeling/01_model_development.ipynb - ML model training")
    print(f"   • Use CLI tool: 'python fpl_tool.py analyze' for quick insights")

else:
    print("❌ No FPL data loaded")
    print("\n🔧 Troubleshooting:")
    print("   1. Run: python ../../fpl_tool.py collect")
    print("   2. Check database path: data/organized/2025-26/")
    print("   3. Verify data collection completed successfully")
    print("   4. Re-run this notebook")

print(f"\n🎯 Ready for FPL 2025/26 Season Planning & Prediction!")

🎯 FPL 2025/26 Season Planning & Analysis Summary

📊 Training Data Status (2024/25 Historical):
   • Players tracked: 740
   • Gameweeks collected: 4
   • Total data points: 2,960
   • Data collection date: September 18, 2025

🎯 Analysis Focus - 2025/26 Season:
   • Using 2024/25 data for model training
   • Building prediction models for 2025/26
   • Historical patterns for future planning

⚽ Performance Insights (Historical 2024/25):
   • Average points per game: 3.03
   • Highest single score: 38
   • Total blanks: 1,384 (46.8%)

🎯 Squad Distribution (Training Data):
   • GKP: 344 records
   • DEF: 980 records
   • MID: 1,312 records
   • FWD: 324 records

✅ Data Quality Assessment:
   • Latest gameweek: 4
   • Historical data ready for 2025/26 modeling

🚀 Next Steps for 2025/26 Season Planning:
   1. Run feature engineering on historical data
   2. Develop predictive models for 2025/26 player performance
   3. Create transfer recommendation system for new season
   4. Build captain 